In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import tensorflow.keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import random
from datetime import datetime
import time

pi = 3.14159265359

maxval=1e9
minval=1e-9

2025-05-13 02:49:34.963822: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-13 02:49:35.879972: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from dataloaders.OptimizedDataGeneratorNew import OptimizedDataGenerator
from loss import *
from models import *

In [3]:
seed = 10
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
dataset_base_dir = "/data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_50x12P5_parquets/contained/recon3D"
tfrecords_base_dir = os.path.join(dataset_base_dir, "TFR_files", "20t")

dataset_train_dir = os.path.join(dataset_base_dir, "train")
dataset_validation_dir = os.path.join(dataset_base_dir, "validation")
tfrecords_dir_train = os.path.join(tfrecords_base_dir, "TFR_train")
tfrecords_dir_val   = os.path.join(tfrecords_base_dir, "TFR_val")

In [5]:
print(f'Number of training files: {len(os.listdir(dataset_train_dir))}')
print(f'Number of validation files: {len(os.listdir(dataset_validation_dir))}')

Number of training files: 80
Number of validation files: 20


In [6]:
batch_size = 5000
val_batch_size = 5000
train_file_size = len(os.listdir(dataset_train_dir))
val_file_size = len(os.listdir(dataset_validation_dir))

In [ ]:
start_time = time.time()
validation_generator = OptimizedDataGenerator(
    dataset_base_dir = dataset_base_dir,
    file_type = "parquet",
    data_format = "3D",
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    input_shape = (20,13,21), # (20,13,21),
    transpose = (0,2,3,1),
    shuffle = False, 
    files_from_end=True,

    tfrecords_dir = tfrecords_dir_val,
    use_time_stamps = -1
    max_workers = 2
)

print("--- Validation generator %s seconds ---" % (time.time() - start_time))


In [ ]:
# training generator
start_time = time.time()
training_generator = OptimizedDataGenerator(
    dataset_base_dir = dataset_base_dir,
    file_type = "parquet",
    data_format = "3D",
    batch_size = batch_size,
    file_count = train_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    input_shape = (20,13,21), # (20,13,21),
    transpose = (0,2,3,1),
    shuffle = False, # True 

    tfrecords_dir = tfrecords_dir_train,
    use_time_stamps = -1
    max_workers = 2
)
print("--- Training generator %s seconds ---" % (time.time() - start_time))

In [7]:
training_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir = tfrecords_dir_train,
    shuffle = True,
    seed = seed,
    quantize = True
)

validation_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir = tfrecords_dir_val,
    shuffle = True,
    seed = seed,
    quantize = True
)


In [8]:
model=CreateModel((13,21,20),n_filters=5,pool_size=3)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=custom_loss
)

model.summary()

2025-05-13 02:49:37.896990: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3234 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 1g.5gb, pci bus id: 0000:01:00.0, compute capability: 8.0


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 21, 20)]      0         
                                                                 
 q_separable_conv2d (QSepar  (None, 11, 19, 5)         285       
 ableConv2D)                                                     
                                                                 
 q_activation (QActivation)  (None, 11, 19, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 11, 19, 5)         30        
                                                                 
 q_activation_1 (QActivatio  (None, 11, 19, 5)         0         
 n)                                                              
                                                                 
 average_pooling2d (Average  (None, 3, 6, 5)           0     

In [9]:
# training
pitch = '50x12P5'
fingerprint = '%08x' % random.randrange(16**8)
base_dir = '/home/dajiang/smart-pixels-ml/weights/weights_7pitches/dataset_3sr_contained_weights/'
weights_dir = base_dir + 'weights-{}-bs{}-{}-20t-checkpoints'.format(pitch, batch_size, fingerprint)

# create output directories
if os.path.isdir(base_dir):
    os.mkdir(weights_dir)
else:
    os.mkdir(base_dir)
    os.mkdir(weights_dir)
    
checkpoint_filepath = weights_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'
mcp = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=False,
)

print('Model fingerprint: {}'.format(fingerprint))

Model fingerprint: 34c2da80


In [ ]:
history = model.fit(x=training_generator,
                    validation_data=validation_generator,
                    callbacks=[mcp],
                    epochs=1000,
                    shuffle=False, # shuffling now occurs within the data-loader
                    verbose=1)

Epoch 1/1000


2025-05-13 02:49:41.316301: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906
2025-05-13 02:49:41.435123: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-05-13 02:49:41.707539: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-05-13 02:49:41.714337: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x76b8600
2025-05-13 02:49:41.736834: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd2a9e78a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-05-13 02:49:41.736880: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB MIG 1g.5gb, Compute Capability 8.0
2025-05-13 02:49:41.741682: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir

93/93 [==============================] - 50s 501ms/step - loss: 45374.3984 - val_loss: 16655.3438
Epoch 2/1000
93/93 [==============================] - 18s 190ms/step - loss: 14994.3350 - val_loss: 13628.5146
Epoch 3/1000
93/93 [==============================] - 18s 193ms/step - loss: 13014.4141 - val_loss: 12182.2773
Epoch 4/1000
93/93 [==============================] - 18s 194ms/step - loss: 11757.8311 - val_loss: 11092.9785
Epoch 5/1000
93/93 [==============================] - 17s 187ms/step - loss: 10758.3623 - val_loss: 10128.8066
Epoch 6/1000
93/93 [==============================] - 18s 193ms/step - loss: 9718.2861 - val_loss: 9201.3984
Epoch 7/1000
93/93 [==============================] - 18s 188ms/step - loss: 9117.0586 - val_loss: 8864.3027
Epoch 8/1000
93/93 [==============================] - 18s 189ms/step - loss: 8899.9287 - val_loss: 8833.4541
Epoch 9/1000
93/93 [==============================] - 18s 191ms/step - loss: 8748.3584 - val_loss: 8665.1797
Epoch 10/1000
93/93 [=